## Quick start
- Run the `pip install` cell.
- Get a free API key from [Google AI Studio](https://aistudio.google.com/app/apikey).
- Set your `GOOGLE_API_KEY` in the code below.

In [13]:
# Install dependencies
%pip install -q -U google-generativeai ipython


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import difflib
from IPython.display import display, HTML
import google.generativeai as genai

# Setup Gemini API
# Get your free key: https://aistudio.google.com/app/apikey
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") or "PASTE_YOUR_KEY_HERE"

if GOOGLE_API_KEY == "PASTE_YOUR_KEY_HERE":
    print("⚠️ Please paste your Google API Key in the variable above.")
else:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✅ Gemini API configured")

✅ Gemini API configured


## Create mock medical reports with errors
We will define sample radiology reports containing spelling mistakes and logical contradictions.

In [15]:
reports_with_errors = [
    """
EXAMINATION: CHEST X-RAY

HISTORY: 45-year-old male with cough and feever.

FINDINGS:
The trakea is midline. The heart size is normal. 
There is a small opacity in the left lower lobe. 
No pleural effusion or pneumothorax.
Osseous structures are intact.

IMPRESSION:
1. Right lower llobe opacity, likely pneumonia.
2. No acute bony abnormality.
""",
    """
EXAMINATION: MRI KNEE LEFT

HISTORY: Twisting injury during soccer game.

FINDINGS:
Therre is a complete tear of the anterior cruciate ligament (ACL). 
The posterior cruciale lickament is intact. 
The medinal meniscus shows a complex tear in the posterior horn.
The lateral meniscus is unremarkable.
There is a large joint effusion.

IMPRESSION:
1. ACL tear.
2. Lateral meniscus tear.
3. Joint effusion.
"""
]

for i, report in enumerate(reports_with_errors):
    print(f"--- Report {i+1} (Original) ---")
    print(report)
    print("\n")

--- Report 1 (Original) ---

EXAMINATION: CHEST X-RAY

HISTORY: 45-year-old male with cough and feever.

FINDINGS:
The trakea is midline. The heart size is normal. 
There is a small opacity in the left lower lobe. 
No pleural effusion or pneumothorax.
Osseous structures are intact.

IMPRESSION:
1. Right lower llobe opacity, likely pneumonia.
2. No acute bony abnormality.



--- Report 2 (Original) ---

EXAMINATION: MRI KNEE LEFT

HISTORY: Twisting injury during soccer game.

FINDINGS:
Therre is a complete tear of the anterior cruciate ligament (ACL). 
The posterior cruciale lickament is intact. 
The medinal meniscus shows a complex tear in the posterior horn.
The lateral meniscus is unremarkable.
There is a large joint effusion.

IMPRESSION:
1. ACL tear.
2. Lateral meniscus tear.
3. Joint effusion.





## Task: Identify and Correct Errors
We will ask the model to analyze the radiology report, identify errors, and provide a corrected version.

In [16]:
model = genai.GenerativeModel('models/gemini-flash-lite-latest')
# or, if you want to use the latest SOTA model
# But limited availability on free tier 
# model = genai.GenerativeModel('models/gemini-3-pro-preview')

prompt_correction = """
You are a helpful medical assistant. Your task is to identify errors in radiology reports.
Look for:
1. Spelling errors.
2. Logical inconsistencies (e.g., findings in the 'left' lung but impression says 'right').

Please provide:
1. A list of the errors found.
2. The fully corrected report text. Do not include any markdown formatting or extra text in the corrected report, just the raw text.
"""

def show_diff(original, corrected):
    d = difflib.HtmlDiff()
    # make_table generates just the table, without the legend
    table = d.make_table(
        original.splitlines(), 
        corrected.splitlines(), 
        fromdesc='Original',
        todesc='Corrected'
    )
    # We need to manually add the styles that make_file usually adds
    html_content = f"<style>{d._styles}</style>{table}"
    display(HTML(html_content))

for i, report in enumerate(reports_with_errors):
    print(f"--- Processing Report {i+1} ---")
    
    # Ask Gemini to find errors and correct them
    response = model.generate_content(f"{prompt_correction}\n\nReport:\n{report}")
    print(response.text)
    
    # Extract the corrected report for the diff
    # Note: This simple extraction assumes the model follows instructions well.
    # In a production app, you'd want more robust parsing (e.g., asking for JSON).
    lines = response.text.split('\n')
    corrected_report_lines = []
    capture = False
    for line in lines:
        if "EXAMINATION:" in line: # Heuristic to find start of report
            capture = True
        if capture:
            corrected_report_lines.append(line)
    
    corrected_text = "\n".join(corrected_report_lines)
    
    if corrected_text:
        print("\n--- Diff View ---")
        show_diff(report, corrected_text)
    else:
        print("Could not automatically extract corrected text for diff view.")
    
    print("\n" + "="*50 + "\n")

--- Processing Report 1 ---
1. Errors Found:
Spelling error: "feever" should be "fever".
Spelling error: "trakea" should be "trachea".
Logical inconsistency: Findings state "left lower lobe" opacity, but Impression states "Right lower llobe" opacity. (Assuming the finding is correct and the impression should match).
Spelling error: "llobe" should be "lobe".

2. Corrected Report:
EXAMINATION: CHEST X-RAY

HISTORY: 45-year-old male with cough and fever.

FINDINGS:
The trachea is midline. The heart size is normal.
There is a small opacity in the left lower lobe.
No pleural effusion or pneumothorax.
Osseous structures are intact.

IMPRESSION:
1. Left lower lobe opacity, likely pneumonia.
2. No acute bony abnormality.

--- Diff View ---
1. Errors Found:
Spelling error: "feever" should be "fever".
Spelling error: "trakea" should be "trachea".
Logical inconsistency: Findings state "left lower lobe" opacity, but Impression states "Right lower llobe" opacity. (Assuming the finding is correct an



--- Processing Report 2 ---
Errors Found:
1. Spelling error: "Therre" should be "There".
2. Spelling error: "cruciale" should be "cruciate".
3. Spelling error: "medinal" should be "medial".
4. Logical inconsistency/Error: The findings state the medial meniscus has a complex tear, but the impression lists a "Lateral meniscus tear."

Corrected Report:
EXAMINATION: MRI KNEE LEFT

HISTORY: Twisting injury during soccer game.

FINDINGS:
There is a complete tear of the anterior cruciate ligament (ACL). 
The posterior cruciate ligament is intact. 
The medial meniscus shows a complex tear in the posterior horn.
The lateral meniscus is unremarkable.
There is a large joint effusion.

IMPRESSION:
1. ACL tear.
2. Medial meniscus tear.
3. Joint effusion.

--- Diff View ---
Errors Found:
1. Spelling error: "Therre" should be "There".
2. Spelling error: "cruciale" should be "cruciate".
3. Spelling error: "medinal" should be "medial".
4. Logical inconsistency/Error: The findings state the medial men